start this notebook in the directory where you want to create the folder structure

you need a folder called "target_structures" with the SLG, BLG and TLG .xyz files in it

then run the runjobs script to automatically run the simulations

# General file functions

In [ ]:
import errno
import os
import shutil


def mkdir_p(path):
    try:
        os.makedirs(path)
    except OSError as exc: # Python > 2.5
        if exc.errno == errno.EEXIST and os.path.isdir(path):
            pass
        else: raise

def safe_open(path):
    mkdir_p(os.path.dirname(path))
    return open(path, 'w')


# Job batch

In [ ]:
# gamma parameters
alist = [0.0105,]
blist = [2.8,]
clist = [2]

# inital angle of the ion
chi = [0, 10, 20]

for target in ['SLG', 'BLG', 'TLG']:
    for q in [30,]:
        for ke in [10, 66, 100]:
            s = f'{target}_Xe{q}_{ke}keV'
            for a in alist:
                for b in blist:
                    for c in clist:
                        for angle in chi:
                            # write input file
                            astring = str(a).replace('.', 'p')
                            bstring = str(b).replace('.', 'p')
                            cstring = str(c).replace('.', 'p')
                            jobname = f'{s}_g-{astring}-{bstring}-{cstring}-a-{angle}'
                            targetfname = f'{target}_tdpot.xyz'
                            print(jobname)
                            dir = f'prod/{target}/Xe{q}_{ke}keV/{jobname}/'
                            fname = dir + 'indat.in'
                            with safe_open(fname) as fo:
                                fo.write(f'{jobname}                 ! jobname\n')
                                fo.write(f'hollow-krc                ! potential\n')
                                fo.write(f'Xe                        ! ion\n')
                                fo.write(f'54 129 {ke} {q}           ! Z, mass, KE, qin\n')
                                fo.write(f'1                         ! ff\n')
                                fo.write(f'0.0 0.0 1 2               ! fwhm_qout, sigma_therm, frozen_par, alpha_max\n')
                                fo.write(f'-30 30 0.01, 0.0001 40000 ! ion_zi, ion_zf, dx_step, acc, nions\n')
                                fo.write(f'{angle} {angle}           ! chi_min, chi_max\n')
                                fo.write(f'{a} {b} {c} 15.0          ! gam_a, gam_b, gam_c, gam_cut\n')
                                fo.write(f'0 0                       ! logmode, print_xyz_files\n')
                                fo.write(f'{targetfname}             ! target filename')

                            # copy in target structure
                            shutil.copyfile(f'target_structures/{targetfname}', dir + targetfname)

                            # write jobscript
                            jobfile = dir + 'job.sh'
                            nodes = 2
                            taks_per_node = 48
                            np  = nodes * taks_per_node
                            with safe_open(jobfile) as fo:
                                fo.write(f'#!/bin/bash\n')
                                fo.write(f'#\n')
                                fo.write(f'#SBATCH -J {jobname}\n')
                                fo.write(f'#SBATCH -N {nodes}\n')
                                fo.write(f'#SBATCH --ntasks-per-node={taks_per_node}\n')
                                fo.write(f'#SBATCH --time=01:00:00\n')
                                fo.write(f'#SBATCH --ntasks-per-core=1\n')
                                fo.write(f'\n')
                                fo.write(f'module load openmpi/4.1.4-gcc-12.2.0-c7xovdh\n')
                                fo.write(f'exe=/home/fs71431/essletzbich/TDpot_fort/TDpot\n')
                                fo.write(f'\n')
                                fo.write(f'mpirun -np {np} $exe')
